In [18]:
import pandas as pd
import requests
import os
import time

from tqdm.auto import tqdm  # for notebooks

tqdm.pandas()

import warnings

from warnings import simplefilter

simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

dev = True

In [19]:
games = pd.DataFrame()

file_path = "replays.parquet"
if os.path.exists(file_path):
    disk = pd.read_parquet(file_path)
    if "id" in disk.columns:
        disk = disk.set_index("id")
    games = disk


n_api_new_rows = limit = 20
page = 0
while n_api_new_rows == limit and limit > 0 and page <= 3:
    page += 1
    apiUrl = f"https://api.bar-rts.com/replays?limit={limit}&preset=team&hasBots=true&page={page}"
    if page > 1:
        time.sleep(1.2)
    json = requests.get(apiUrl, headers={"User-Agent": "tetrisface"}).json()

    data = json["data"]

    api = pd.DataFrame.from_records(data).set_index("id")

    api_new_indices = api.index.difference(games.index)
    n_api_new_rows = len(api_new_indices)
    n_before_games = len(games)
    games = pd.concat(
        [games, api.loc[api_new_indices]],
        verify_integrity=True,
        axis=0,
    )
    games.startTime = pd.to_datetime(games.startTime)
    games.durationMs = pd.to_numeric(games.durationMs, downcast="integer")
    games.drop(["Map"], axis=1, errors="ignore", inplace=True)
    print(f"games {n_before_games} + {n_api_new_rows} = {len(games)}")
games.to_parquet(file_path)

games 15527 + 14 = 15541


In [20]:
pd.options.mode.copy_on_write = True


def is_raptors(row):
    for team in row:
        for ai in team["AIs"]:
            if ai["shortName"] == "RaptorsAI":
                return True
    return False


def is_draw(row):
    results = []
    for team in row:
        results.append(team["winningTeam"])
    return len(team) <= 1 or all(
        x == results[0] for x in [team["winningTeam"] for team in row]
    )


def winners(row):
    _winners = []
    for team in row:
        if team["winningTeam"] is True:
            if len(team["Players"]) > 0:
                _winners.extend([player["name"] for player in team["Players"]])
            elif len(team["AIs"]) > 0:
                _winners.extend([ai["shortName"] for ai in team["AIs"]])
    return _winners


def players(row):
    _players = []
    for team in row:
        _players.extend([player["name"] for player in team["Players"]])
    return _players


games["raptors"] = games["AllyTeams"].apply(is_raptors)
games["draw"] = games["AllyTeams"].apply(is_draw)
games["winners"] = games["AllyTeams"].apply(winners)
games["players"] = games["AllyTeams"].apply(players)

In [21]:
numerical_columns = [
    "ai_incomemultiplier",
    "air_rework",
    "allowpausegameplay",
    "allowuserwidgets",
    "april1",
    "april1extra",
    "assistdronesair",
    "assistdronesbuildpowermultiplier",
    "assistdronescount",
    "capturebonus",
    "captureradius",
    "capturetime",
    "commanderbuildersbuildpower",
    "commanderbuildersrange",
    "coop",
    "critters",
    "debugcommands",
    "decapspeed",
    "defaultdecals",
    "disable_fogofwar",
    "disablemapdamage",
    "dominationscore",
    "dominationscoretime",
    "easter_egg_hunt",
    "easteregghunt",
    "emprework",
    "energyperpoint",
    "experimentalextraunits",
    "experimentalimprovedtransports",
    "experimentallegionfaction",
    "experimentalmassoverride",
    "experimentalnoaircollisions",
    "experimentalrebalancet2energy",
    "experimentalrebalancet2labs",
    "experimentalrebalancet2metalextractors",
    "experimentalrebalancewreckstandarization",
    "experimentalreversegear",
    "experimentalxpgain",
    "faction_limiter",
    "ffa_wreckage",
    "fixedallies",
    "lategame_rebalance",
    "limitscore",
    "map_atmosphere",
    "map_waterislava",
    "map_waterlevel",
    "maxunits",
    "metalperpoint",
    "multiplier_builddistance",
    "multiplier_buildpower",
    "multiplier_buildtimecost",
    "multiplier_energyconversion",
    "multiplier_energycost",
    "multiplier_energyproduction",
    "multiplier_losrange",
    "multiplier_maxdamage",
    "multiplier_maxvelocity",
    "multiplier_metalcost",
    "multiplier_metalextraction",
    "multiplier_radarrange",
    "multiplier_resourceincome",
    "multiplier_shieldpower",
    "multiplier_turnrate",
    "multiplier_weapondamage",
    "multiplier_weaponrange",
    "norush",
    "norushtimer",
    "numberofcontrolpoints",
    "proposed_unit_reworks",
    "ranked_game",
    "raptor_endless",
    "raptor_firstwavesboost",
    "raptor_graceperiodmult",
    "raptor_queentimemult",
    "raptor_spawncountmult",
    "raptor_spawntimemult",
    "releasecandidates",
    "ruins_civilian_disable",
    "ruins_only_t1",
    "scav_bosstimemult",
    "scav_endless",
    "scav_graceperiodmult",
    "scav_spawncountmult",
    "scav_spawntimemult",
    "scoremode_chess_adduptime",
    "scoremode_chess_spawnsperphase",
    "scoremode_chess_unbalanced",
    "scoremode_chess",
    "shareddynamicalliancevictory",
    "skyshift",
    "startenergy",
    "startenergystorage",
    "startmetal",
    "startmetalstorage",
    "starttime",
    "teamffa_start_boxes_shuffle",
    "tugofwarmodifier",
    "unified_maxslope",
    "unit_restrictions_noair",
    "unit_restrictions_noconverters",
    "unit_restrictions_noendgamelrpc",
    "unit_restrictions_noextractors",
    "unit_restrictions_nolrpc",
    "unit_restrictions_nonukes",
    "unit_restrictions_notacnukes",
    "unit_restrictions_notech2",
    "unit_restrictions_notech3",
    "usemapconfig",
    "usemexconfig",
]
string_columns = [
    "assistdronesenabled",
    "commanderbuildersenabled",
    "deathmode",
    "experimentalshields",
    "experimentalstandardgravity",
    "lootboxes_density",
    "lootboxes",
    "map_tidal",
    "raptor_difficulty",
    "raptor_raptorstart",
    "ruins_density",
    "ruins",
    "scav_difficulty",
    "scav_scavstart",
    "scoremode",
    "teamcolors_anonymous_mode",
    "teamcolors_icon_dev_mode",
    "transportenemy",
    "tweakdefs",
    "tweakdefs1",
    "tweakdefs2",
    "tweakdefs3",
    "tweakdefs4",
    "tweakdefs5",
    "tweakdefs6",
    "tweakdefs7",
    "tweakdefs8",
    "tweakdefs9",
    "tweakunits",
    "tweakunits1",
    "tweakunits2",
    "tweakunits3",
    "tweakunits4",
    "tweakunits5",
    "tweakunits6",
    "tweakunits7",
    "tweakunits8",
    "tweakunits9",
]


def cast_frame(_df):

    for col in string_columns:
        _df[string_columns] = _df[string_columns].fillna("")

    _df = _df.astype({col: str for col in string_columns}, errors="raise")

    for col in numerical_columns:
        _df[numerical_columns] = _df[numerical_columns].fillna(0)

    for col in numerical_columns:
        _df[col] = pd.to_numeric(
            _df[col],
            downcast="integer",
        )

    return _df

In [22]:
raptor_games = games[
    games["raptors"]
    # & ~df_root_expanded["draw"]
]  # draws might be good to exclude

raptor_games["fetch_success"] = None


def api_replay_detail(row):
    time.sleep(1.2)
    if row is not None and row.name is not None:
        url = f"https://api.bar-rts.com/replays/{row.name}"
        response = requests.get(url, headers={"User-Agent": "tetrisface"})
        if response.status_code == 200:
            response_json = response.json()
            replay_details = response_json.get("gameSettings")
            replay_details["awards"] = response_json.get("awards")
            replay_details["AllyTeams"] = response_json.get("AllyTeams")
            for key, value in replay_details.items():
                # Add new column to DataFrame if the column doesn't exist
                if key not in raptor_games.columns:
                    raptor_games[key] = None
                # Update DataFrame with fetched data
                row[key] = value
            row["fetch_success"] = True
            return row
    print(f"Failed to fetch data from {url}")
    row["fetch_success"] = False
    return row


# load in disk
game_detail_path = "replays_gamesettings.parquet"
if os.path.exists(game_detail_path):
    disk = pd.read_parquet(game_detail_path)
    with warnings.catch_warnings():
        warnings.simplefilter(action="ignore", category=FutureWarning)

    raptor_games.loc[disk.index, disk.columns] = disk


isnull = raptor_games[
    raptor_games.fetch_success.isnull() | (raptor_games.fetch_success == False)
]
to_fetch = isnull.head(100)
print(f"fetching {len(to_fetch)} of {len(isnull)} missing games")

# fetch new
df_raptors_api = to_fetch.progress_apply(
    api_replay_detail,
    axis=1,
)
raptor_games.loc[df_raptors_api.index, df_raptors_api.columns] = cast_frame(
    df_raptors_api
)

if len(raptor_games.loc[raptor_games.fetch_success == False]) > 0:
    print(f"failed to fetch {len(raptor_games[~raptor_games.fetch_success])} games")

raptor_games = cast_frame(raptor_games)

fetching 2 of 2 missing games


/tmp/ipykernel_13004/787936207.py:39: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[2992167 1506867 2103033 ... 3200433  865000  536600]' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  raptor_games.loc[disk.index, disk.columns] = disk


  0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
raptor_games.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Index: 2881 entries, 0cb40a66f0eb04a7025bc929bf299f35 to c69d12664038c7684ddeeeda447f0a91
Data columns (total 157 columns):
 #    Column                                    Dtype              
---   ------                                    -----              
 0    startTime                                 datetime64[ns, UTC]
 1    durationMs                                int64              
 2    AllyTeams                                 object             
 3    raptors                                   bool               
 4    draw                                      bool               
 5    winners                                   object             
 6    players                                   object             
 7    fetch_success                             object             
 8    Map                                       object             
 9    transportenemy                            object             
 10   tweakunits3     

In [24]:
# refetch all
# raptor_games["fetch_success"] = False
# store
raptor_games[raptor_games["fetch_success"].notnull()].to_parquet(game_detail_path)

In [25]:
from nuttyb import hp_multiplier, main, coms, meganuke_149, wind_restrict_149, gamesettings

raptor_games['raptor_win'] = raptor_games.apply(lambda row: 'RaptorsAI' in row['winners'] and (row['draw'] == False), axis=1)
raptor_games['player_win'] = raptor_games.apply(lambda row: ('RaptorsAI' not in row['winners'])
                                                and (len(row['winners']) > 0)
                                                and (row['draw'] == False), axis=1)



raptor_games["nuttyb_main"] = raptor_games.apply(
    lambda row: all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.48"
        ]
    )
    or all(
        [
            tweak["value"] == row[tweak["location"]]
            for tweak in main
            if tweak["version"] == "1.49"
        ]
    ),
    axis=1,
)


def nuttyb_difficulty(row):
    for _def in hp_multiplier:
        if 'values' in _def:
            if row[_def["location"]] in _def["values"]:
                return _def["name"]
        else:
            if row[_def["location"]] == _def["value"]:
                return _def["name"]
    return None


raptor_games["nuttyb_hp"] = raptor_games.apply(nuttyb_difficulty, axis=1)


possible_tweaks = ['tweakunits', 'tweakdefs'] +[f'tweakunits{i}' for i in range(1, 10)] + [f'tweakdefs{i}' for i in range(1, 10)]
all_allowed_tweaks = []
for setting_dict in [setting_dict for setting_dict in [*main, *hp_multiplier, *coms, meganuke_149, wind_restrict_149]]:
    if 'values' in setting_dict:
        all_allowed_tweaks.extend(setting_dict['values'])
    else:
        all_allowed_tweaks.append(setting_dict['value'])

def is_default_nuttyb_tweaks(row):
    if row["nuttyb_main"] == True \
        and row["nuttyb_hp"] is not None and len(row["nuttyb_hp"]) > 0 \
        and row[possible_tweaks][row[possible_tweaks].astype(bool)].isin(all_allowed_tweaks).all():
        #     and row['NuttyB Mode'] is not None and len(row['NuttyB Mode']) > 0
        return True
    return False

raptor_games["default_nuttyb_tweaks"] = raptor_games.apply(is_default_nuttyb_tweaks, axis=1)


def raptor_diff(row):
    if row["default_nuttyb_tweaks"]:
        return f"NuttyB Default {row['nuttyb_hp']}"
    elif row["nuttyb_main"]:
        if row["nuttyb_hp"]:
            return f"NuttyB Main & HP {row["nuttyb_hp"]}"
        else:
            return f"NuttyB Main & {row["raptor_difficulty"]}"
    elif row["nuttyb_hp"]:
        return f"NuttyB HP {row["nuttyb_hp"]}"
    else:
        return f"{row["raptor_difficulty"]}"

raptor_games["Difficulty"] = pd.Categorical(
    raptor_games.apply(
        raptor_diff,
        axis=1),
        [
            "NuttyB Default Epicest",
            "NuttyB Default Epicer+",
            "NuttyB Default Epic++",
            "NuttyB Default Epic+",
            "NuttyB Default Epic",
            "NuttyB Main & HP Epicest",
            "NuttyB Main & HP Epicer+",
            "NuttyB Main & HP Epic++",
            "NuttyB Main & HP Epic+",
            "NuttyB Main & HP Epic",
            "NuttyB HP Epicest",
            "NuttyB HP Epicer+",
            "NuttyB HP Epic++",
            "NuttyB HP Epic+",
            "NuttyB HP Epic",
            "NuttyB Main & epic",
            "NuttyB Main & veryhard",
            "NuttyB Main & hard",
            "NuttyB Main & normal",
            "NuttyB Main & easy",
            "NuttyB Main & veryeasy",
            "epic",
            "veryhard",
            "hard",
            "normal",
            "easy",
            "veryeasy",
        ],
        ordered=True,
)

raptor_games['raptor_raptorstart'] = pd.Categorical(
    raptor_games['raptor_raptorstart'],
    ['alwaysbox', 'initialbox', 'avoid'],
    ordered=True
)
higher_harder = {
    'raptor_spawncountmult',
    'raptor_firstwavesboost',
    # 'raptor_raptorstart', # uncertain
}
lower_harder = {
    'startmetal',
    'startenergy',
    'startmetalstorage',
    'startenergystorage',
    'multiplier_builddistance',
    'multiplier_shieldpower',
    'multiplier_buildpower',
    'commanderbuildersrange',
    'commanderbuildersbuildpower',
    'raptor_queentimemult', # probably harder
    'raptor_spawntimemult',
}

def nuttyb_mode(row):
    for mode_name, settings in gamesettings.items():
        match = False
        # print(f'checking mode {mode_name}, {settings}')
        for setting, value in settings.items():
            if row[setting] == value or (setting in higher_harder and row[setting] >= value) or (setting == 'raptor_raptorstart' and row[setting] == 'avoid') or (setting in lower_harder and row[setting] <= value):
                # print(f'value matching mode {mode_name} {setting} {row[setting]} ~= {value} higher harder {setting in higher_harder} lower harder {setting in lower_harder}')
                match = True
            elif row[setting] != value:
                # print(f'value not matching mode {mode_name} {setting} {row[setting]} != {value}')
                match = False
                break
            else:
                raise Exception(f"unhandled setting {setting} value {value}")
        if match:
            return mode_name
    return None
# print(f'found mode { nuttyb_mode(raptor_games.loc['5bd8116605ce75f19618af055dd363a4']) }')
# print(f'found mode { nuttyb_mode(raptor_games.loc['46990c66b597eee2b2d25216620d652b']) }')
raptor_games["NuttyB Mode"] = pd.Categorical(
    raptor_games.apply(nuttyb_mode, axis=1),
    [
        "Gauntlet",
        "0 grace zerg",
        "Zerg",
        "0 grace",
        "Rush",
        "Gauntlet 1.48",
        "Zerg 1.48",
        "Rush 1.48",
    ],
    ordered=True
)

with pd.option_context("display.max_rows", None, "display.max_columns", None):
    try:
        na_games = raptor_games[raptor_games["Difficulty"].isna() & raptor_games["fetch_success"] == True]
        na_games_related = na_games[['nuttyb_main', "nuttyb_hp",'raptor_difficulty', 'Difficulty', 'NuttyB Mode']]
        assert (
            len(na_games) == 0
        ), f'missing difficulties for {len(na_games)} {na_games_related}'
    except AssertionError as e:
        print(e)
        print(na_games_related)

In [51]:
def awards(row):
    player_team_id = None
    for ally_team in row["AllyTeams"]:
        if len(ally_team["Players"]) > 0:
            for player in ally_team["Players"]:
                if player["name"] == row["player"] and "teamId" in player:
                    player_team_id = player["teamId"]
                    break

    if player_team_id is None or not row["awards"]:
        return pd.Series([None, None])

    damage = None
    eco = None
    try:
        if player_team_id == row["awards"]["fightingUnitsDestroyed"][0]["teamId"]:
            damage = 1
    except KeyError:
        damage = 0

    try:
        if player_team_id == row["awards"]["mostResourcesProduced"]["teamId"]:
            eco = 1
    except KeyError:
        eco = 0

    return pd.Series([damage, eco])


def links_cell(url_pairs):
    cell = "=" + f'ifna(hyperlink("{url_pairs[0][0]}";"[{url_pairs[0][1]+1}]")'
    if len(url_pairs) > 1:
        cell += (
            '; "'
            + ", ".join([f"{index+1}: {url}" for url, index in url_pairs[1:]])
            + '"'
        )
    return cell + ")"


def grouped(_df):
    all_groups = pd.DataFrame()
    for (group_diff, group_mode), group_df in _df.groupby(
        ["Difficulty", "NuttyB Mode"], observed=True, sort=True
    ):
        # print(group_mode, group_diff)
        # continue
        group_df["Replays"] = group_df.index
        group_df["_player_group_bypass"] = group_df["player"]
        group_players = (
            group_df.groupby(["player"])
            .agg(
                {
                    "player": "count",
                    "award_damage": "sum",
                    "award_eco": "sum",
                    "Replays": lambda replay_ids: links_cell(
                        [
                            (f"https://bar-rts.com/replays/{replay_id}", index)
                            for index, replay_id in list(
                                reversed(list(enumerate(replay_ids)))
                            )
                        ]
                    ),
                    "_player_group_bypass": lambda x: x.iloc[0],
                }
            )
            .rename(
                columns={
                    "Replays": "Victories",
                    "_player_group_bypass": "Player",
                    "award_damage": "🏆DMG",
                    "award_eco": "🏆ECO",
                },
            )
            .reset_index(drop=True)
            .sort_values("player", ascending=False)
            .reset_index(drop=True)[["Player", "Victories", "🏆DMG", "🏆ECO"]]
        )
        # group_players["NuttyB Mode"] = group_mode
        # group_players["Difficulty"] = group_diff

        group_players.columns = pd.MultiIndex.from_tuples(
            [
                (f"{group_diff} - {group_mode}", second_level_columns)
                for second_level_columns in group_players.columns
            ]
        )
        print(f"{group_diff} - {group_mode}")

        all_groups = (
            pd.concat(
                [
                    all_groups,
                    group_players,
                    # [
                    #     [
                    #         "Difficulty",
                    #         "NuttyB Mode",
                    #         "Player",
                    #         "Victories",
                    #         "🏆DMG",
                    #         "🏆ECO",
                    #     ]
                    # ],
                ],
                axis=1,
            )
            .fillna("")
            .replace(0, "")
        )

    return all_groups


import datetime
import gspread

gc = gspread.service_account()

# spreadsheet = gc.open_by_key("1oI7EJIUiwLLXDMBgky2BN8gM6eaQRb9poWGiP2IKot0")
spreadsheet = gc.open_by_key("1w8Ng9GGUo6DU0rBFRYRnC_JxK8nHTSSjf1Nbq1e-3bc")  # dev


def update_sheet(_df, sheet_name_postfix=" NuttyB"):
    sheet_name = datetime.date.today().strftime("%Y-%m") + sheet_name_postfix
    new_sheet = False
    try:
        worksheet = spreadsheet.worksheet(sheet_name)
        worksheet.clear()
    except gspread.exceptions.WorksheetNotFound:
        worksheet = spreadsheet.add_worksheet(
            title=sheet_name, rows=len(_df), cols=len(_df.columns), index=0
        )
        new_sheet = True

    first_header, second_header = zip(*_df.columns)
    values = (
        [[x if index % 4 == 0 else "" for index, x in enumerate(first_header)]]
        + [second_header]
        + _df.values.tolist()
    )
    worksheet.update(
        values=values,
        value_input_option=gspread.utils.ValueInputOption.user_entered,
    )

    unicode_char_columns = [index for index, x in enumerate(second_header) if "🏆" in x]
    player_columns = [index for index, x in enumerate(second_header) if "Player" in x]

    sheet_id = worksheet._properties["sheetId"]
    body = {
        "requests": [
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet_id,
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "textFormat": {
                                "fontSize": 11,
                            },
                            "hyperlinkDisplayType": "LINKED",
                        }
                    },
                    "fields": "userEnteredFormat",
                }
            },
            {
                "autoResizeDimensions": {
                    "dimensions": {
                        "sheetId": sheet_id,
                        "dimension": "COLUMNS",
                    }
                }
            },
            {
                "repeatCell": {
                    "range": {
                        "sheetId": sheet_id,
                    },
                    "cell": {
                        "userEnteredFormat": {
                            "textFormat": {
                                "fontSize": 10,
                            },
                            "hyperlinkDisplayType": "LINKED",
                        }
                    },
                    "fields": "userEnteredFormat",
                }
            },
            {
                "updateSheetProperties": {
                    "properties": {
                        "sheetId": sheet_id,
                        "gridProperties": {"frozenRowCount": 2},
                    },
                    "fields": "gridProperties(frozenRowCount)",
                }
            },
        ]
        + [
            {
                "updateDimensionProperties": {
                    "range": {
                        "sheetId": sheet_id,
                        "dimension": "COLUMNS",
                        "startIndex": index,
                        "endIndex": index + 1,
                    },
                    "properties": {"pixelSize": 52},
                    "fields": "pixelSize",
                }
            }
            for index in unicode_char_columns
        ]
        + [
            {
                "repeatCell": {
                    "cell": {
                        "userEnteredFormat": {
                            "horizontalAlignment": "LEFT",
                        }
                    },
                    "range": {
                        "sheetId": sheet_id,
                        "startColumnIndex": index,
                        "endColumnIndex": index + 1,
                    },
                    "fields": "userEnteredFormat(horizontalAlignment)",
                }
            }
            for index in player_columns
        ]
    }
    if new_sheet:
        body["requests"].append(
            {
                "addBanding": {
                    "bandedRange": {
                        "bandedRangeId": sheet_id,
                        "range": {
                            "sheetId": sheet_id,
                            "startRowIndex": 2,
                        },
                        "rowProperties": {
                            "firstBandColorStyle": {
                                "rgbColor": {
                                    "red": 243 / 255,
                                    "green": 243 / 255,
                                    "blue": 243 / 255,
                                },
                            },
                            "secondBandColorStyle": {"themeColor": "BACKGROUND"},
                        },
                    },
                },
            }
        )
    res = spreadsheet.batch_update(body)


raptor_players = raptor_games.explode("players")
raptor_players.rename(columns={"players": "player"}, inplace=True)

raptor_players["won"] = raptor_players.apply(
    lambda row: row.player in row.winners, axis=1
)

raptor_players[["award_damage", "award_eco"]] = raptor_players.apply(awards, axis=1)

all_winners = grouped(raptor_players[raptor_players["player_win"]])
default_nuttyb_winners = grouped(
    raptor_players[
        (raptor_players["player_win"])
        & (raptor_players["default_nuttyb_tweaks"] == True)
    ]
)

update_sheet(all_winners, " All")
update_sheet(default_nuttyb_winners, " NuttyB")

with pd.option_context(
    "display.max_rows",
    None,
    "display.max_columns",
    None,
    "display.float_format",
    "{:.0f}".format,
):
    display(all_winners.head(20))
    display(default_nuttyb_winners.head(20))
# update_sheet(grouped(default_nuttyb_winners))

NuttyB Default Epic+ - 0 grace
NuttyB Default Epic+ - Rush
NuttyB Default Epic+ - Rush 1.48
NuttyB Main & HP Epic++ - Gauntlet 1.48
NuttyB Main & HP Epic+ - Rush 1.48
NuttyB Main & HP Epic - Gauntlet 1.48
NuttyB Main & HP Epic - Rush 1.48
NuttyB HP Epic+ - 0 grace
NuttyB HP Epic+ - Rush
NuttyB HP Epic+ - Rush 1.48
NuttyB HP Epic - Gauntlet 1.48
NuttyB Main & epic - 0 grace
NuttyB Main & epic - Rush 1.48
NuttyB Default Epic+ - 0 grace
NuttyB Default Epic+ - Rush
NuttyB Default Epic+ - Rush 1.48


NuttyB Default Epic+ - 0 grace  \
                           Player   
0                       974625813   
1                        Backbash   
2                         myriari   
3                         hueskii   
4                       HotPotato   
5                          Bucket   
6                    ArmCommander   
7                  BathroomPrince   
8                          Bogdan   
9                     DeadlyPants   
10                    CptSquishyy   
11                      CNCPlayer   
12                        DockFam   
13                           Efan   
14                         Erlias   
15                   DoinkDynasty   
16                        DogMeat   
17                   ImaginaryFox   
18                     LangesRohr   
19           Notes_of_the_Archive   

                                                                 \
                                            Victories 🏆DMG 🏆ECO   
0   =ifna(hyperlink("https://bar-rts.com/replays/d...             
1   =ifna(hyperlink("https://bar-rts.com/replays/d...    2    2   
2   =ifna(hyperlink("https://bar-rts.com/replays/d...             
3   =ifna(hyperlink("https://bar-rts.com/replays/1...             
4   =ifna(hyperlink("https://bar-rts.com/replays/d...             
5   =ifna(hyperlink("https://bar-rts.com/replays/2...             
6   =ifna(hyperlink("https://bar-rts.com/replays/2...             
7   =ifna(hyperlink("https://bar-rts.com/replays/1...             
8   =ifna(hyperlink("https://bar-rts.com/replays/2...             
9   =ifna(hyperlink("https://bar-rts.com/replays/d...             
10  =ifna(hyperlink("https://bar-rts.com/replays/d...             
11  =ifna(hyperlink("https://bar-rts.com/replays/1...             
12  =ifna(hyperlink("https://bar-rts.com/replays/1...             
13  =ifna(hyperlink("https://bar-rts.com/replays/1...             
14  =ifna(hyperlink("https://bar-rts.com/replays/1...             
15  =ifna(hyperlink("https://bar-rts.com/replays/2...             
16  =ifna(hyperlink("https://bar-rts.com/replays/2...             
17  =ifna(hyperlink("https://bar-rts.com/replays/1...             
18  =ifna(hyperlink("https://bar-rts.com/replays/1...             
19  =ifna(hyperlink("https://bar-rts.com/replays/d...             

   NuttyB Default Epic+ - Rush  \
                        Player   
0                      myriari   
1                    HotPotato   
2                    Smash4321   
3                      berayen   
4                     Backbash   
5                       iMobIt   
6                    974625813   
7                         4UDO   
8                   Alexthemad   
9                   19chiodowi   
10                      Finray   
11                 Draginosaur   
12                    ChronoBR   
13                ArmCommander   
14                  AirElite03   
15                  Insatiable   
16                    Fyreworm   
17                   FirstCore   
18                  JunoMiNha1   
19                       Kenno   

                                                                 \
                                            Victories 🏆DMG 🏆ECO   
0   =ifna(hyperlink("https://bar-rts.com/replays/9...    2    1   
1   =ifna(hyperlink("https://bar-rts.com/replays/9...         1   
2   =ifna(hyperlink("https://bar-rts.com/replays/9...             
3   =ifna(hyperlink("https://bar-rts.com/replays/9...             
4   =ifna(hyperlink("https://bar-rts.com/replays/1...    2    2   
5   =ifna(hyperlink("https://bar-rts.com/replays/9...             
6   =ifna(hyperlink("https://bar-rts.com/replays/9...             
7   =ifna(hyperlink("https://bar-rts.com/replays/9...             
8   =ifna(hyperlink("https://bar-rts.com/replays/9...             
9   =ifna(hyperlink("https://bar-rts.com/replays/1...             
10  =ifna(hyperlink("https://bar-rts.com/replays/1...             
11  =ifna(hyperlink("https://bar-rts.com/replays/1...             
12  =ifna(hyperlink("https

NuttyB Default Epic+ - 0 grace  \
                           Player   
0                       974625813   
1                        Backbash   
2                         myriari   
3                         hueskii   
4                       HotPotato   
5                          Bucket   
6                    ArmCommander   
7                  BathroomPrince   
8                          Bogdan   
9                     DeadlyPants   
10                    CptSquishyy   
11                      CNCPlayer   
12                        DockFam   
13                           Efan   
14                         Erlias   
15                   DoinkDynasty   
16                        DogMeat   
17                   ImaginaryFox   
18                     LangesRohr   
19           Notes_of_the_Archive   

                                                                 \
                                            Victories 🏆DMG 🏆ECO   
0   =ifna(hyperlink("https://bar-rts.com/replays/d...             
1   =ifna(hyperlink("https://bar-rts.com/replays/d...    2    2   
2   =ifna(hyperlink("https://bar-rts.com/replays/d...             
3   =ifna(hyperlink("https://bar-rts.com/replays/1...             
4   =ifna(hyperlink("https://bar-rts.com/replays/d...             
5   =ifna(hyperlink("https://bar-rts.com/replays/2...             
6   =ifna(hyperlink("https://bar-rts.com/replays/2...             
7   =ifna(hyperlink("https://bar-rts.com/replays/1...             
8   =ifna(hyperlink("https://bar-rts.com/replays/2...             
9   =ifna(hyperlink("https://bar-rts.com/replays/d...             
10  =ifna(hyperlink("https://bar-rts.com/replays/d...             
11  =ifna(hyperlink("https://bar-rts.com/replays/1...             
12  =ifna(hyperlink("https://bar-rts.com/replays/1...             
13  =ifna(hyperlink("https://bar-rts.com/replays/1...             
14  =ifna(hyperlink("https://bar-rts.com/replays/1...             
15  =ifna(hyperlink("https://bar-rts.com/replays/2...             
16  =ifna(hyperlink("https://bar-rts.com/replays/2...             
17  =ifna(hyperlink("https://bar-rts.com/replays/1...             
18  =ifna(hyperlink("https://bar-rts.com/replays/1...             
19  =ifna(hyperlink("https://bar-rts.com/replays/d...             

   NuttyB Default Epic+ - Rush  \
                        Player   
0                      myriari   
1                    HotPotato   
2                    Smash4321   
3                      berayen   
4                     Backbash   
5                       iMobIt   
6                    974625813   
7                         4UDO   
8                   Alexthemad   
9                   19chiodowi   
10                      Finray   
11                 Draginosaur   
12                    ChronoBR   
13                ArmCommander   
14                  AirElite03   
15                  Insatiable   
16                    Fyreworm   
17                   FirstCore   
18                  JunoMiNha1   
19                       Kenno   

                                                                 \
                                            Victories 🏆DMG 🏆ECO   
0   =ifna(hyperlink("https://bar-rts.com/replays/9...    2    1   
1   =ifna(hyperlink("https://bar-rts.com/replays/9...         1   
2   =ifna(hyperlink("https://bar-rts.com/replays/9...             
3   =ifna(hyperlink("https://bar-rts.com/replays/9...             
4   =ifna(hyperlink("https://bar-rts.com/replays/1...    2    2   
5   =ifna(hyperlink("https://bar-rts.com/replays/9...             
6   =ifna(hyperlink("https://bar-rts.com/replays/9...             
7   =ifna(hyperlink("https://bar-rts.com/replays/9...             
8   =ifna(hyperlink("https://bar-rts.com/replays/9...             
9   =ifna(hyperlink("https://bar-rts.com/replays/1...             
10  =ifna(hyperlink("https://bar-rts.com/replays/1...             
11  =ifna(hyperlink("https://bar-rts.com/replays/1...             
12  =ifna(hyperlink("https

In [59]:
raptor_games.groupby("NuttyB Mode", observed=True).size()
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(raptor_games.loc["aa441066266c75370bc7591ab0f84630"])

NuttyB Mode
0 grace          57
Rush             44
Gauntlet 1.48    20
Rush 1.48        81
dtype: int64

In [60]:
# print(f'nuttyb main {len(raptor_games[raptor_games["nuttyb_main"]])}')
# print(
#     f'nuttyb main + hp {len(raptor_games[raptor_games["nuttyb_main"] & raptor_games["nuttyb_hp"]])}'
# )
# print(
#     f'default nuttyb {len(raptor_games[raptor_games["default_nuttyb_tweaks"] & raptor_games["NuttyB Mode"].notna()])}'
# )
# default_wins = raptor_games[
#     (raptor_games["default_nuttyb_tweaks"] == True)
#     & (raptor_games["player_win"] == True)
#     & (raptor_games["NuttyB Mode"].notna())
# ]
# print(f"default nuttyb wins {len(default_wins)}")
# with pd.option_context(
#     "display.max_rows", None, "display.max_columns", None, "display.max_colwidth", None
# ):
#     display(
#         default_wins[["startTime", "Difficulty", "NuttyB Mode", "players", "winners"]]
#     )

In [61]:
import csv


# player_game_mask = raptor_games["players"].apply(lambda x: 'tetrisface' in x)
player_game_mask = False

total_unique = raptor_players.player.nunique()

print(
    f'{len(raptor_games[player_game_mask] if player_game_mask else raptor_games)} games and {total_unique} player names between {raptor_players["startTime"].min().date()} and {raptor_players["startTime"].max().date()}'
)
# raptor_players["Difficulty"] =
agg_total = raptor_players.groupby(["Difficulty"], observed=True).agg(
    {"player": "nunique"}
)
agg_won = (
    raptor_players[raptor_players["won"]]
    .groupby(["Difficulty"], observed=True)
    .agg({"player": "nunique"})
)
game_modes = agg_total.join(agg_won, lsuffix="s have tried", rsuffix="s have won")
game_modes["% have won"] = (
    game_modes["players have won"] / game_modes["players have tried"] * 100
)
game_modes["% tried of total"] = game_modes["players have tried"] / total_unique * 100
game_modes["% won of total"] = game_modes["players have won"] / total_unique * 100
game_modes.fillna(0, inplace=True)
game_modes["players have won"] = game_modes["players have won"].astype(int)
# game_modes.to_csv("game_modes_stats.csv", quoting=csv.QUOTE_ALL)
display(
    game_modes.style.format(
        {
            "% have won": "{:.0f}%",
            "% tried of total": "{:.0f}%",
            "% won of total": "{:.0f}%",
        }
    ).set_table_styles(
        [{"selector": ".row_heading", "props": [("text-align", "left")]}]
    )
)

2840 games and 3028 player names between 2024-03-10 and 2024-04-07


,players have tried,players have won,% have won,% tried of total,% won of total
Difficulty,,,,,
veryeasy,47,18,38%,2%,1%
easy,94,59,63%,3%,2%
normal,982,420,43%,32%,14%
hard,403,134,33%,13%,4%
veryhard,624,114,18%,21%,4%
epic,822,251,31%,27%,8%
NuttyB HP Epic,48,20,42%,2%,1%
NuttyB HP Epic+,585,268,46%,19%,9%
NuttyB HP Epic++,20,0,0%,1%,0%


In [62]:
from math import log2
from collections import Counter


def ID3_entropies(data_df):
    """
    https://gist.github.com/whitehaven/bbd408edca38de93637635b52d2bba89

    Takes pandas.DataFrame and returns a series with all non-index schemas' entropies calculated.

    It supports non-binary field types by calculating average entropy. Result series starts with the most productive decision level.
    """

    def entropy_for_field(field):
        entropy = 0
        field_entry_count = len(field)

        # get count of unique
        field_counter = Counter(field)

        # E( Si/S * E(pi*log2(pi)) )
        for trait, count in field_counter.items():
            p_T = count / field_entry_count
            p_F = (field_entry_count - count) / field_entry_count

            if p_T == 0 or p_F == 0:
                entropy = 0
                break
            # Si/S * E(pi*log2(pi))
            entropy += (
                count / field_entry_count * (-(p_T * log2(p_T)) - (p_F * log2(p_F)))
            )
        return entropy

    data_df_entropy = {}
    for field in data_df:
        entropy_this_field = entropy_for_field(data_df[field])
        data_df_entropy[field] = entropy_this_field

    data_df_entropy_se = pd.Series(data_df_entropy)
    data_df_entropy_se.sort_values(inplace=True, ascending=False)
    return data_df_entropy_se


df_entropy = ID3_entropies(
    raptor_games.loc[:, raptor_games.columns.isin(numerical_columns)]
).to_frame("entropy")

In [63]:
stats_nuttyb = raptor_games[
    (raptor_games["nuttyb_main"] == True) | raptor_games["nuttyb_hp"] == True
].loc[:, raptor_games.columns.isin(numerical_columns)]
stats_nuttyb_entropy = ID3_entropies(stats_nuttyb).to_frame("entropy")

entropy_joined = stats_nuttyb.agg(
    ["mean", "median", "min", "max", "std", "skew", "kurt"]
).T.join(stats_nuttyb_entropy, how="outer")
entropy_joined = entropy_joined[
    (entropy_joined["entropy"] > 0) & (entropy_joined["min"] != entropy_joined["max"])
]
with pd.option_context("display.max_rows", 500):
    display(
        entropy_joined.sort_values(by="entropy", ascending=False)
        .style.format(
            {
                "median": "{:.1f}",
                "min": "{:.1f}",
                "max": "{:.1f}",
                "mean": "{:.2f}",
                "std": "{:.2f}",
                "skew": "{:.2f}",
                "kurt": "{:.2f}",
                "entropy": "{:.2f}",
            }
        )
        .set_table_styles(
            [{"selector": ".row_heading", "props": [("text-align", "left")]}]
        )
    )

,mean,median,min,max,std,skew,kurt,entropy
experimentalrebalancewreckstandarization,0.62,1.0,0.0,1.0,0.49,-0.50,-1.75,0.96
norushtimer,1.68,0.0,0.0,5.0,2.36,0.69,-1.52,0.92
ranked_game,0.69,1.0,0.0,1.0,0.46,-0.80,-1.36,0.90
disablemapdamage,0.70,1.0,0.0,1.0,0.46,-0.89,-1.21,0.88
decapspeed,0.58,0.0,0.0,2.0,0.91,0.94,-1.12,0.87
scoremode_chess,0.29,0.0,0.0,1.0,0.45,0.94,-1.12,0.87
usemapconfig,0.29,0.0,0.0,1.0,0.45,0.94,-1.12,0.87
dominationscoretime,8.64,0.0,0.0,30.0,13.59,0.94,-1.12,0.87
numberofcontrolpoints,3.74,0.0,0.0,13.0,5.89,0.94,-1.12,0.87
limitscore,86.41,0.0,0.0,300.0,135.93,0.94,-1.12,0.87


In [64]:
# df_raptors = df_raptors.explode("players")
# df_raptors.rename(columns={"players": "player"}, inplace=True)
# df_raptors["won"] = df_raptors.apply(lambda row: row.player in row.winners, axis=1)

# df_raptors["week"] = pd.to_datetime(df_raptors["startTime"]).dt.month
# df = df_raptors.groupby(["player", "week"]).agg({"won": "mean", "id": "count"})
# df = df.rename(columns={"won": "winrate", "id": "replays"})

# df = df[df.replays > 4]

# rank_sum_column = "rank_sum"
# df[rank_sum_column] = (df.winrate.rank() * 0.1 + df.replays.rank() * 0.9) / len(df)
# df.sort_values(rank_sum_column, inplace=True, ascending=False)

# df[rank_sum_column] = round(df[rank_sum_column], 2).astype(str)

# df.winrate = df.winrate * 100

In [65]:
# pd.set_option("display.max_rows", 500)
# pd.options.display.float_format = "{:.0f}%".format

# for week, df_week in df.groupby("week"):
#     # df_week = df_week[df_week["replays"] > 4].reset_index()
#     df_week = df_week.reset_index()
#     df_week.index = np.arange(1, len(df_week) + 1)
#     print(df_week.head(20))
#     print()